<a href="https://colab.research.google.com/github/adautofbn/ri_labs/blob/master/lab06/modelo_vetorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [299]:
import pandas as pd
import numpy as np
import nltk
import re
import collections
import bisect
from nltk.tokenize import RegexpTokenizer

nltk.download('stopwords')
result = pd.read_csv('https://raw.githubusercontent.com/adautofbn/ri_labs/master/lab06/results.csv')
                                            # Resultados adquiridos do site brasil-247 em abril de 2019

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 1. Reconstruir o índice considerando o conjunto de dados que indicamos. Esses são os dados coletados por Bernardi e os estaremos usando para facilitar a correção da atividade. Se você já estiver usando esses dados não precisa reconstruir o índice;

In [0]:
tknz = RegexpTokenizer(r'([A-Za-zÁáÉéÍíÓóÚúÃãÕõÇçÂâÊê]{3,27})')
stopwords = nltk.corpus.stopwords.words('portuguese') 
indexes = {}
n = 0

for text in result.text:
  words = [word for word in tknz.tokenize(text.lower())
           if not bool(re.search(r'\d', word))
           and word not in stopwords and len(word) >= 3]  
  n += 1
  for t in words:
    if t not in indexes.keys():
      indexes[t] = []
    indexes[t].append(n)
    
for elem in indexes.items():
  d = dict(collections.Counter(elem[1]))
  indexes[elem[0]] = list(d.items())

## 2. Refinar o índice invertido de forma a também incluir o IDF (inverse document frequency) de cada termo do dicionário; 

In [0]:
M = result.text.count()
for word in indexes:
  k = len(indexes[word])
  IDF = round(np.log((M+1)/k),2)
  indexes[word].append(IDF)

## 3. Implementar as seguintes versões do modelo vetorial:

### 3.1. Representação binária

In [0]:
def binary_vsm(query, document):
  score = 0
  query_tokens = query.split()
  doc_tokens = document.split()
  
  for token in query_tokens:
    score += (token in doc_tokens)
    
  return score

### 3.2. TF (lembre-se que esse modelo já está implementado)

In [0]:
def tf_vsm(query, document):
  score = 0
  doc_tokens = document.split()
  query_tokens = query.split()
  
  for word in query_tokens:
    score += doc_tokens.count(word)
  
  return score

### 3.3 TF-IDF

In [0]:
def tfidf_vsm(query, document):
  score = 0
  doc_tokens = document.split()
  query_tokens = query.split()
  
  for word in query_tokens:
    cwd = doc_tokens.count(word)
    score += cwd * indexes[word][-1]
  
  return round(score,2)

### 3.4 BM25 (não usaremos Okapi já que os documentos não tem grande variação de tamanho)

In [0]:
def bm25_vsm(query, document, k):
  score = 0
  doc_tokens = document.split()
  query_tokens = query.split()
  
  words = [word for word in query_tokens if word in doc_tokens]
    
  for word in words:
    cwd = doc_tokens.count(word)
    dfw = len(indexes[word][:-1])
    score += (((k+1) * cwd) / (cwd + k)) * np.log10(((M+1) / dfw))
  
  return round(score,2)

## 4. Execute os algoritmos separadamente em 3 consultas de sua escolha e retorne os top-5 documentos mais similares à cada consulta;

In [0]:
queries = ['jair bolsonaro', 'damares alves', 'paulo guedes']

In [0]:
def create_top5_models(query):
  n = 0
  db = []
  dtf = []
  dtfidf = []
  dbm25 = []
  for doc in result.text:
    doc = doc.lower()
    n += 1
    bisect.insort(db, (binary_vsm(query, doc), n))
    bisect.insort(dtf, (tf_vsm(query,doc), n))
    bisect.insort(dtfidf, (tfidf_vsm(query,doc), n))
    bisect.insort(dbm25, (bm25_vsm(query,doc,10), n))
  
  db.reverse()
  dtf.reverse()
  dtfidf.reverse()
  dbm25.reverse()
  
  return db[:5], dtf[:5], dtfidf[:5], dbm25[:5]

In [0]:
top5_binary = ['','','']
top5_tf = ['','','']
top5_tfidf = ['','','']
top5_bm25 = ['','','']

top5_binary[0], top5_tf[0], top5_tfidf[0], top5_bm25[0] = create_top5_models(queries[0])
top5_binary[1], top5_tf[1], top5_tfidf[1], top5_bm25[1] = create_top5_models(queries[1])
top5_binary[2], top5_tf[2], top5_tfidf[2], top5_bm25[2] = create_top5_models(queries[2])

In [309]:
query_df = pd.DataFrame()

query_df['Query'] = queries
query_df['Binary'] = top5_binary
query_df['TF'] = top5_tf
query_df['TF-IDF'] = top5_tfidf
query_df['BM25'] = top5_bm25

query_df.index+=1
query_df


,Query,Binary,TF,TF-IDF,BM25
1,jair bolsonaro,"[(2, 238), (2, 237), (2, 231), (2, 224), (2, 2...","[(41, 207), (37, 151), (31, 166), (19, 19), (1...","[(67.98, 207), (55.14, 151), (42.18, 166), (25...","[(11.17, 207), (8.9, 151), (5.84, 166), (4.41,..."
2,damares alves,"[(2, 151), (2, 18), (1, 213), (1, 7), (0, 249)]","[(2, 151), (2, 18), (1, 213), (1, 7), (0, 249)]","[(8.84, 151), (8.84, 18), (4.42, 213), (4.42, ...","[(3.84, 151), (3.84, 18), (1.92, 213), (1.92, ..."
3,paulo guedes,"[(2, 240), (2, 236), (2, 235), (2, 218), (2, 2...","[(9, 240), (9, 209), (9, 37), (6, 213), (5, 223)]","[(24.54, 37), (23.16, 240), (20.4, 209), (15.9...","[(6.86, 240), (6.77, 37), (6.63, 209), (5.24, ..."


## 5. Compare os resultados encontrados e responda.

### 5.1. Quais modelos você acha que trouxe os melhores resultados? Por que? Inspecione os documentos retornados para melhor embasar sua resposta.

### 5.2. Calcule e reporte o overlap par-a-par entre os resultados de cada modelo (usando o índice de Jaccard)